In [1]:
%reload_ext tensorboard
import numpy as np
import json
import tensorflow as tf
import os

import time
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import gym

import tensorflow as tf

gpus = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(gpus[0], True)
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(gpus[0], 
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=9000)])
  except RuntimeError as e:
    print(e)
    
    
print("GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))
tf.config.experimental.list_logical_devices('GPU')

import tf_agents
from tf_agents.environments import py_environment, parallel_py_environment, batched_py_environment
from tf_agents.environments import tf_environment
from tf_agents.environments import tf_py_environment
from tf_agents.environments import utils
from tf_agents.specs import array_spec
from tf_agents.environments import wrappers
from tf_agents.environments import suite_gym
from tf_agents.trajectories import time_step as ts
from tf_agents.networks.q_network import QNetwork
from tf_agents.agents.dqn.dqn_agent import DqnAgent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.metrics import tf_metrics
from tf_agents.drivers.dynamic_step_driver import DynamicStepDriver
from tf_agents.policies.random_tf_policy import RandomTFPolicy
from tf_agents.utils.common import function, element_wise_squared_loss
from tf_agents.eval.metric_utils import log_metrics
from tf_agents.policies import policy_saver
import logging

import tensorflow.keras as keras

tf.compat.v1.enable_v2_behavior()
import time
import json
import datetime
import copy
import shutil

#import imp
from scan_gym import envs
#imp.reload(envs)


seed=42
tf.random.set_seed(seed)
np.random.seed(seed)

GPUs:  1


In [2]:
current_path = os.getcwd()
params_file = os.path.join(current_path, 'params.json') 
pm=json.load(open(params_file))
run_label = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
data_log_path = os.path.join(current_path, 'generated_data/') 

In [3]:
data_paths = ['/home/pico/uni/romi/rl_sony/arabidopsis_image_sets/000',
              '/home/pico/uni/romi/rl_sony/arabidopsis_image_sets/001',
              '/home/pico/uni/romi/rl_sony/arabidopsis_image_sets/003',
              '/home/pico/uni/romi/rl_sony/arabidopsis_image_sets/006']

envs = [suite_gym.load('ScannerEnv-v1',gym_kwargs={'dataset_path':dp}) 
        for dp in data_paths]

tf_env = tf_py_environment.TFPyEnvironment( batched_py_environment.BatchedPyEnvironment(envs,multithreading=True))#, isolation=True  )

/home/pico/.local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [4]:
tf_env.observation_spec()

(BoundedTensorSpec(shape=(82, 82, 2), dtype=tf.float32, name='observation/tuple_0', minimum=array(0., dtype=float32), maximum=array(255., dtype=float32)),
 BoundedTensorSpec(shape=(5,), dtype=tf.float32, name='observation/tuple_1', minimum=array([-400.,    0.,    0.,  -20.,    0.], dtype=float32), maximum=array([400.,   8.,  20.,  20.,   1.], dtype=float32)))

In [5]:
def image_layers():
    input_img = keras.layers.Input(shape=(82,82,2))
    '''x = keras.layers.Lambda(lambda x: x * 0)(input_img)
    x = keras.layers.Conv2D(filters=1,
                            kernel_size=1,
                            padding="same",
                            strides=(82,82),
                            activation='relu')(x)'''
    
    preprocessing = keras.layers.Lambda(lambda x: x / 255.)(input_img)
    x = keras.layers.Conv2D(filters=32,
                            kernel_size=8,
                            padding="same",
                            strides=(4,4),
                            activation='relu')(preprocessing)
    x = keras.layers.Conv2D(filters=64,
                            kernel_size=4,
                            padding="same",
                            strides=(2,2),
                            activation='relu')(x)
    x = keras.layers.Conv2D(filters=64,
                            kernel_size=3,
                            padding="same",
                            strides=(1,1),
                            activation='relu')(x)
    x = keras.layers.Flatten()(x)
    model = keras.models.Model(inputs=input_img,outputs=x)
    return model
    
    
    
#scale range 0 to 1
oldmin = tf_env.observation_spec()[0].minimum
oldmax = tf_env.observation_spec()[0].maximum
    
def input_vect_layers():
    input_ = keras.layers.Input(shape=(5,))
    preprocessing = keras.layers.Lambda(lambda x: ((x-oldmin)*(1.- 0.)/(oldmax-oldmin)) + 0. )(input_)
    #x = keras.layers.Dense(4)(preprocessing)
    return keras.models.Model(inputs=input_,outputs=preprocessing)

In [6]:
#network
preprocessing_layers=(image_layers(),input_vect_layers())
preprocessing_combiner = tf.keras.layers.Concatenate(axis=-1)
dense_l = pm['model']['fc_layer_params']
if len(dense_l) == 1:
    fc_layer_params = (dense_l[0],)
else:
    fc_layer_params = dense_l

q_net = QNetwork(
tf_env.observation_spec(),
tf_env.action_spec(),
preprocessing_layers=preprocessing_layers,
preprocessing_combiner=preprocessing_combiner,
fc_layer_params=fc_layer_params)

WARNING - 2021-04-13 20:09:30,011 - encoding_network - Beware: Copying a layer that has already been built: 'model'.  This can lead to subtle bugs because the original layer's weights will not be used in the copy.
WARNING - 2021-04-13 20:09:30,049 - encoding_network - Beware: Copying a layer that has already been built: 'model_1'.  This can lead to subtle bugs because the original layer's weights will not be used in the copy.


In [7]:
#agent
train_step = tf.Variable(0)
#optimizer = keras.optimizers.RMSprop(lr=2.5e-4, rho=0.95, momentum=0.0,
#            epsilon=0.00001, centered=True)
'''lr_decay =  keras.optimizers.schedules.PolynomialDecay(
            initial_learning_rate=0.005, # initial ε
            decay_steps = pm['agent']['decay_steps'], 
            end_learning_rate=0.0005)
optimizer = keras.optimizers.Adam(learning_rate=lambda: lr_decay(train_step))'''


optimizer = keras.optimizers.Adam(learning_rate=pm['model']['learning_rate'])

epsilon_fn = keras.optimizers.schedules.PolynomialDecay(
            initial_learning_rate=1.0, # initial ε
            decay_steps = pm['agent']['decay_steps'], 
            end_learning_rate=0.01) # final ε

agent = DqnAgent(tf_env.time_step_spec(),
                tf_env.action_spec(),
                q_network=q_net,
                optimizer=optimizer,
                target_update_period=pm['agent']['target_update_period'],
                td_errors_loss_fn= element_wise_squared_loss, #keras.losses.Huber(reduction="none"),
                gamma=pm['agent']['gamma'], # discount factor
                train_step_counter=train_step,
                epsilon_greedy=lambda: epsilon_fn(train_step))
agent.initialize()


WARNING - 2021-04-13 20:09:42,284 - encoding_network - Beware: Copying a layer that has already been built: 'model'.  This can lead to subtle bugs because the original layer's weights will not be used in the copy.
WARNING - 2021-04-13 20:09:42,303 - encoding_network - Beware: Copying a layer that has already been built: 'model_1'.  This can lead to subtle bugs because the original layer's weights will not be used in the copy.


In [8]:
#Replay buffer
replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec= agent.collect_data_spec,
    batch_size= tf_env.batch_size,
    max_length=pm['rbuffer']['max_length'])

In [9]:
#observer
#observer is just a function (or a callable object) that takes a trajectory argument,
#add_method() method (bound to the replay_buffer object) can be used as observer
replay_buffer_observer = replay_buffer.add_batch

In [10]:
#observer for training metrics
training_metrics = [
tf_metrics.NumberOfEpisodes(),
tf_metrics.AverageEpisodeLengthMetric(batch_size=len(envs)),
tf_metrics.EnvironmentSteps(),
tf_metrics.AverageReturnMetric(batch_size=len(envs)),
tf_metrics.MaxReturnMetric(batch_size=len(envs)),
tf_metrics.MinReturnMetric(batch_size=len(envs)),   
]


'''#observer for training metrics
training_metrics = [
tf_metrics.NumberOfEpisodes(),
tf_metrics.AverageEpisodeLengthMetric(),
tf_metrics.EnvironmentSteps(),
tf_metrics.AverageReturnMetric(),
tf_metrics.MaxReturnMetric(),
tf_metrics.MinReturnMetric(),   
]'''

'#observer for training metrics\ntraining_metrics = [\ntf_metrics.NumberOfEpisodes(),\ntf_metrics.AverageEpisodeLengthMetric(),\ntf_metrics.EnvironmentSteps(),\ntf_metrics.AverageReturnMetric(),\ntf_metrics.MaxReturnMetric(),\ntf_metrics.MinReturnMetric(),   \n]'

In [11]:
#custom observer
class ShowProgress:
    def __init__(self, total):
        self.counter = 0
        self.total = total
    def __call__(self, trajectory):
        if not trajectory.is_boundary():
            self.counter += 1
        if self.counter % 100 == 0:
            print("\r{}/{}".format(self.counter, self.total), end="")

In [12]:
#Collect Driver
update_period = pm['collect_driver']['num_steps'] # train the model every x steps
collect_driver = DynamicStepDriver(
    tf_env,
    agent.collect_policy,
    observers=[replay_buffer_observer] + training_metrics,
    num_steps=update_period) # collect x steps for each training iteration

#+ training_metrics,

In [13]:
# random policy driver to start filling the buffer
random_collect_policy = RandomTFPolicy(tf_env.time_step_spec(),
                        tf_env.action_spec())

ns = pm['rnd_policy']['num_steps']
init_driver = DynamicStepDriver(
            tf_env,
            random_collect_policy,
            observers=[replay_buffer.add_batch],
            num_steps=ns)
#, ShowProgress(ns)],
final_time_step, final_policy_state = init_driver.run()

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


WARNING - 2021-04-13 20:09:43,175 - deprecation - From /home/pico/.local/lib/python3.8/site-packages/tf_agents/drivers/dynamic_step_driver.py:196: calling while_loop_v2 (from tensorflow.python.ops.control_flow_ops) with back_prop=False is deprecated and will be removed in a future version.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.while_loop(c, b, vars, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.while_loop(c, b, vars))


In [14]:
#use buffer as tf API dataset ()
dataset = replay_buffer.as_dataset(
        sample_batch_size=pm['rbuffer']['sample_batch_size'],
        num_steps=2,
        num_parallel_calls=3).prefetch(3)

Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


WARNING - 2021-04-13 20:10:00,467 - api - From /home/pico/anaconda3/envs/rl/lib/python3.8/site-packages/tensorflow/python/autograph/operators/control_flow.py:1218: ReplayBuffer.get_next (from tf_agents.replay_buffers.replay_buffer) is deprecated and will be removed in a future version.
Instructions for updating:
Use `as_dataset(..., single_deterministic_pass=False) instead.


In [15]:
#convert main functions to tensorflow functions to speed up training
collect_driver.run = function(collect_driver.run)
agent.train = function(agent.train)

In [16]:
#@tf.function
def train_agent(n_iterations):
    #reset metrics
    for m in training_metrics:
        m.reset()
    time_step = None
    policy_state = ()#agent.collect_policy.get_initial_state(tf_env.batch_size)
    agent.train_step_counter.assign(0)
    iterator = iter(dataset)
    for iteration in range(n_iterations):
        time_step, policy_state = collect_driver.run(time_step, policy_state)
        trajectories, buffer_info = next(iterator)
        train_loss = agent.train(trajectories)
        print("\r{} loss:{:.5f}".format(iteration, train_loss.loss.numpy()), end="")
        if iteration % 100 == 0:
            with train_summary_writer.as_default():
                #plot metrics
                for train_metric in training_metrics:
                    train_metric.tf_summaries(train_step=tf.cast(agent.train_step_counter,tf.int64), step_metrics=training_metrics[:])
                train_summary_writer.flush()
                
        if iteration % 500 == 0:
            with train_summary_writer.as_default():
                #plot train loss          
                tf.summary.scalar('train_loss', train_loss.loss.numpy(), step=tf.cast(agent.train_step_counter,tf.int64))
                #plot NN weights
                for layer in  q_net.layers:
                    for weight in layer.weights:
                        tf.summary.histogram(weight.name,weight,step=tf.cast(agent.train_step_counter,tf.int64))
                train_summary_writer.flush()

In [17]:
train_dir = os.path.join(data_log_path,"logs/",run_label)   
train_summary_writer = tf.summary.create_file_writer(
            train_dir, flush_millis=10000)
#train_summary_writer.set_as_default()

In [18]:
# Launch TensorBoard with objects in the log directory
# This should launch tensorboard in your browser, but you may not see your metadata.
#%tensorboard --logdir=logs --reload_interval=15

In [19]:
#tf.summary.scalar('avgreturn', training_metrics[3].result().numpy(), step=tf.cast(agent.train_step_counter,tf.int64))

In [20]:
%time train_agent(pm['misc']['n_iterations'])

Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


WARNING - 2021-04-13 20:10:16,826 - deprecation - From /home/pico/anaconda3/envs/rl/lib/python3.8/site-packages/tensorflow/python/util/dispatch.py:201: calling foldr_v2 (from tensorflow.python.ops.functional_ops) with back_prop=False is deprecated and will be removed in a future version.
Instructions for updating:
back_prop=False is deprecated. Consider using tf.stop_gradient instead.
Instead of:
results = tf.foldr(fn, elems, back_prop=False)
Use:
results = tf.nest.map_structure(tf.stop_gradient, tf.foldr(fn, elems))


99999 loss:0.3427033CPU times: user 4h 56min 28s, sys: 33min 13s, total: 5h 29min 41s
Wall time: 4h 21min 28s


In [21]:
#save model, parameters and code state
policy_dir = os.path.join(data_log_path,"policies", run_label)
tf_policy_saver = policy_saver.PolicySaver(agent.policy)
tf_policy_saver.save(policy_dir)

with open(os.path.join(data_log_path,"parameters", run_label+'.json'), 'w') as json_file:
  json.dump(pm, json_file)

src = os.path.join(current_path,"dqn_tf_agents.ipynb")
dst = os.path.join(data_log_path,"train_code", run_label+'.ipynb')
shutil.copyfile(src, dst)

src = os.path.join(current_path,"scan_gym/scan_gym/envs/ScannerEnv/scanner_env.py")
dst = os.path.join(data_log_path,"environment_code", run_label+'.py')
shutil.copyfile(src, dst)


WARNING - 2021-04-14 00:31:30,139 - save - Found untraced functions such as QNetwork_layer_call_fn, QNetwork_layer_call_and_return_conditional_losses, EncodingNetwork_layer_call_fn, EncodingNetwork_layer_call_and_return_conditional_losses, dense_1_layer_call_fn while saving (showing 5 of 30). These functions will not be directly callable after loading.
WARNING - 2021-04-14 00:31:30,172 - save - Found untraced functions such as QNetwork_layer_call_fn, QNetwork_layer_call_and_return_conditional_losses, EncodingNetwork_layer_call_fn, EncodingNetwork_layer_call_and_return_conditional_losses, dense_1_layer_call_fn while saving (showing 5 of 30). These functions will not be directly callable after loading.


INFO:tensorflow:Assets written to: /home/pico/uni/romi/rl_sony/tf-agents_plant_scanner/generated_data/policies/20210413-200828/assets


INFO - 2021-04-14 00:31:30,343 - builder_impl - Assets written to: /home/pico/uni/romi/rl_sony/tf-agents_plant_scanner/generated_data/policies/20210413-200828/assets


'/home/pico/uni/romi/rl_sony/tf-agents_plant_scanner/generated_data/environment_code/20210413-200828.py'

In [22]:
replay_buffer.num_frames()

<tf.Tensor: shape=(), dtype=int64, numpy=200000>

In [23]:
dp = '/home/pico/uni/romi/rl_sony/arabidopsis_image_sets/009'
env = suite_gym.load('ScannerEnv-v1',gym_kwargs={'dataset_path':dp})
test_env = tf_py_environment.TFPyEnvironment( env )

/home/pico/.local/lib/python3.8/site-packages/gym/logger.py:30: UserWarning: WARN: Box bound precision lowered by casting to float32
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [31]:
state = test_env.reset()
#print(state.observation[0].numpy())
last_position = env.current_position
cont = 0
time_steps = 3000
#obs = deque(maxlen=time_steps)

for i in range(time_steps):
    #obs.append(state.observation[0].numpy())
    cont +=1
    action_step = agent.policy.action(state)#agent.policy.action(state)
    next_state = test_env.step(action_step.action)
    print(cont,':',last_position,state.observation[1].numpy(),env.action2move(action_step.action.numpy(),state.observation[1].numpy()[0][-1]) ) #,state.step_type.numpy()  )#,env.action2move(action_step.action,state.observation[0][-1].numpy()))#action,dir
    state = copy.deepcopy(next_state)
    last_position = env.current_position
    if state.is_last():
        print('salio',cont)
        break
        
print(env.kept_images)
print(sorted(env.kept_images))
print(env.total_reward)
print(len(env.kept_images))
print(env.total_moves)
print(env.chamfer_from_collected())

1 : 650 [[0. 1. 0. 0. 0.]] 81111
2 : 650 [[0. 1. 1. 0. 1.]] 81111
3 : 650 [[0. 1. 2. 0. 1.]] 81111
4 : 650 [[0. 1. 3. 0. 1.]] 81111
5 : 650 [[0. 1. 4. 0. 1.]] 81111
6 : 650 [[0. 1. 5. 0. 1.]] 81111
7 : 650 [[0. 1. 6. 0. 1.]] 81111
8 : 650 [[0. 1. 7. 0. 1.]] 81111
9 : 650 [[0. 1. 8. 0. 1.]] 81111
10 : 650 [[0. 1. 9. 0. 1.]] 81111
11 : 650 [[ 0.  1. 10.  0.  1.]] 81111
12 : 650 [[ 0.  1. 11.  0.  1.]] 81111
13 : 650 [[ 0.  1. 12.  0.  1.]] 81111
14 : 650 [[ 0.  1. 13.  0.  1.]] 81111
15 : 650 [[ 0.  1. 14.  0.  1.]] 81111
16 : 650 [[ 0.  1. 15.  0.  1.]] 81111
17 : 650 [[ 0.  1. 16.  0.  1.]] 81111
18 : 650 [[ 0.  1. 17.  0.  1.]] 11118
19 : 650 [[ 0.  1. 18.  0.  0.]] 11118
20 : 650 [[ 0.  1. 19.  0.  0.]] 11118
21 : 650 [[ 0.  1. 20.  0.  0.]] 88888
22 : 650 [[0. 1. 0. 0. 0.]] 81111
23 : 650 [[0. 1. 1. 0. 1.]] 81111
24 : 650 [[0. 1. 2. 0. 1.]] 81111
25 : 650 [[0. 1. 3. 0. 1.]] 81111
26 : 650 [[0. 1. 4. 0. 1.]] 81111
27 : 650 [[0. 1. 5. 0. 1.]] 81111
28 : 650 [[0. 1. 6. 0. 1.]] 81111
29

244 : 650 [[ 0.  1. 12.  0.  1.]] 81111
245 : 650 [[ 0.  1. 13.  0.  1.]] 81111
246 : 650 [[ 0.  1. 14.  0.  1.]] 81111
247 : 650 [[ 0.  1. 15.  0.  1.]] 81111
248 : 650 [[ 0.  1. 16.  0.  1.]] 81111
249 : 650 [[ 0.  1. 17.  0.  1.]] 11118
250 : 650 [[ 0.  1. 18.  0.  0.]] 11118
251 : 650 [[ 0.  1. 19.  0.  0.]] 11118
252 : 650 [[ 0.  1. 20.  0.  0.]] 88888
253 : 650 [[0. 1. 0. 0. 0.]] 81111
254 : 650 [[0. 1. 1. 0. 1.]] 81111
255 : 650 [[0. 1. 2. 0. 1.]] 81111
256 : 650 [[0. 1. 3. 0. 1.]] 81111
257 : 650 [[0. 1. 4. 0. 1.]] 81111
258 : 650 [[0. 1. 5. 0. 1.]] 81111
259 : 650 [[0. 1. 6. 0. 1.]] 81111
260 : 650 [[0. 1. 7. 0. 1.]] 81111
261 : 650 [[0. 1. 8. 0. 1.]] 81111
262 : 650 [[0. 1. 9. 0. 1.]] 81111
263 : 650 [[ 0.  1. 10.  0.  1.]] 81111
264 : 650 [[ 0.  1. 11.  0.  1.]] 81111
265 : 650 [[ 0.  1. 12.  0.  1.]] 81111
266 : 650 [[ 0.  1. 13.  0.  1.]] 81111
267 : 650 [[ 0.  1. 14.  0.  1.]] 81111
268 : 650 [[ 0.  1. 15.  0.  1.]] 81111
269 : 650 [[ 0.  1. 16.  0.  1.]] 81111
270 : 650 

482 : 650 [[ 0.  1. 19.  0.  0.]] 11118
483 : 650 [[ 0.  1. 20.  0.  0.]] 88888
484 : 650 [[0. 1. 0. 0. 0.]] 81111
485 : 650 [[0. 1. 1. 0. 1.]] 81111
486 : 650 [[0. 1. 2. 0. 1.]] 81111
487 : 650 [[0. 1. 3. 0. 1.]] 81111
488 : 650 [[0. 1. 4. 0. 1.]] 81111
489 : 650 [[0. 1. 5. 0. 1.]] 81111
490 : 650 [[0. 1. 6. 0. 1.]] 81111
491 : 650 [[0. 1. 7. 0. 1.]] 81111
492 : 650 [[0. 1. 8. 0. 1.]] 81111
493 : 650 [[0. 1. 9. 0. 1.]] 81111
494 : 650 [[ 0.  1. 10.  0.  1.]] 81111
495 : 650 [[ 0.  1. 11.  0.  1.]] 81111
496 : 650 [[ 0.  1. 12.  0.  1.]] 81111
497 : 650 [[ 0.  1. 13.  0.  1.]] 81111
498 : 650 [[ 0.  1. 14.  0.  1.]] 81111
499 : 650 [[ 0.  1. 15.  0.  1.]] 81111
500 : 650 [[ 0.  1. 16.  0.  1.]] 81111
501 : 650 [[ 0.  1. 17.  0.  1.]] 11118
502 : 650 [[ 0.  1. 18.  0.  0.]] 11118
503 : 650 [[ 0.  1. 19.  0.  0.]] 11118
504 : 650 [[ 0.  1. 20.  0.  0.]] 88888
505 : 650 [[0. 1. 0. 0. 0.]] 81111
506 : 650 [[0. 1. 1. 0. 1.]] 81111
507 : 650 [[0. 1. 2. 0. 1.]] 81111
508 : 650 [[0. 1. 3. 0. 1

719 : 650 [[0. 1. 4. 0. 1.]] 81111
720 : 650 [[0. 1. 5. 0. 1.]] 81111
721 : 650 [[0. 1. 6. 0. 1.]] 81111
722 : 650 [[0. 1. 7. 0. 1.]] 81111
723 : 650 [[0. 1. 8. 0. 1.]] 81111
724 : 650 [[0. 1. 9. 0. 1.]] 81111
725 : 650 [[ 0.  1. 10.  0.  1.]] 81111
726 : 650 [[ 0.  1. 11.  0.  1.]] 81111
727 : 650 [[ 0.  1. 12.  0.  1.]] 81111
728 : 650 [[ 0.  1. 13.  0.  1.]] 81111
729 : 650 [[ 0.  1. 14.  0.  1.]] 81111
730 : 650 [[ 0.  1. 15.  0.  1.]] 81111
731 : 650 [[ 0.  1. 16.  0.  1.]] 81111
732 : 650 [[ 0.  1. 17.  0.  1.]] 11118
733 : 650 [[ 0.  1. 18.  0.  0.]] 11118
734 : 650 [[ 0.  1. 19.  0.  0.]] 11118
735 : 650 [[ 0.  1. 20.  0.  0.]] 88888
736 : 650 [[0. 1. 0. 0. 0.]] 81111
737 : 650 [[0. 1. 1. 0. 1.]] 81111
738 : 650 [[0. 1. 2. 0. 1.]] 81111
739 : 650 [[0. 1. 3. 0. 1.]] 81111
740 : 650 [[0. 1. 4. 0. 1.]] 81111
741 : 650 [[0. 1. 5. 0. 1.]] 81111
742 : 650 [[0. 1. 6. 0. 1.]] 81111
743 : 650 [[0. 1. 7. 0. 1.]] 81111
744 : 650 [[0. 1. 8. 0. 1.]] 81111
745 : 650 [[0. 1. 9. 0. 1.]] 81111


954 : 650 [[0. 1. 8. 0. 1.]] 81111
955 : 650 [[0. 1. 9. 0. 1.]] 81111
956 : 650 [[ 0.  1. 10.  0.  1.]] 81111
957 : 650 [[ 0.  1. 11.  0.  1.]] 81111
958 : 650 [[ 0.  1. 12.  0.  1.]] 81111
959 : 650 [[ 0.  1. 13.  0.  1.]] 81111
960 : 650 [[ 0.  1. 14.  0.  1.]] 81111
961 : 650 [[ 0.  1. 15.  0.  1.]] 81111
962 : 650 [[ 0.  1. 16.  0.  1.]] 81111
963 : 650 [[ 0.  1. 17.  0.  1.]] 11118
964 : 650 [[ 0.  1. 18.  0.  0.]] 11118
965 : 650 [[ 0.  1. 19.  0.  0.]] 11118
966 : 650 [[ 0.  1. 20.  0.  0.]] 88888
967 : 650 [[0. 1. 0. 0. 0.]] 81111
968 : 650 [[0. 1. 1. 0. 1.]] 81111
969 : 650 [[0. 1. 2. 0. 1.]] 81111
970 : 650 [[0. 1. 3. 0. 1.]] 81111
971 : 650 [[0. 1. 4. 0. 1.]] 81111
972 : 650 [[0. 1. 5. 0. 1.]] 81111
973 : 650 [[0. 1. 6. 0. 1.]] 81111
974 : 650 [[0. 1. 7. 0. 1.]] 81111
975 : 650 [[0. 1. 8. 0. 1.]] 81111
976 : 650 [[0. 1. 9. 0. 1.]] 81111
977 : 650 [[ 0.  1. 10.  0.  1.]] 81111
978 : 650 [[ 0.  1. 11.  0.  1.]] 81111
979 : 650 [[ 0.  1. 12.  0.  1.]] 81111
980 : 650 [[ 0.  1. 

In [25]:
#batched_py_environment.BatchedPyEnvironment(envs=[suite.load(...) for _ in range(n)])

In [26]:
#q_net.layers[0].weights[0].shape